In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## First, grab the list of pokemon and their base stats from the web
Bulbapedia has this list

In [2]:
import urllib
import re

In [5]:
def get_matching_line_number(pattern, lines):
    has_match = False
    for i, line in enumerate(lines):
        match = re.search(pattern, line)
        if match:
            has_match = True
            break
    return i if has_match else -1

In [94]:
stats_url = (
    'https://bulbapedia.bulbagarden.net/wiki/'
    'List_of_Pok%C3%A9mon_by_base_stats_in_Generation_I'
)
page = urllib.request.urlopen(stats_url)
html_bytes = page.read()
html = html_bytes.decode('utf-8')
html_lines = html.split('\n')

In [95]:
print(get_matching_line_number(r'\)">[A-Z][^<]*<', html_lines))
print(get_matching_line_number(r'">\d+$', html_lines))
print(get_matching_line_number(r'0029', html_lines))
print(get_matching_line_number(r'0030', html_lines))

546
548
1130
1151


In [96]:
indices = range(1, 152)
bs_dict = {'index': indices, 'name': [], 'base_hp': [], 'base_att': [],
           'base_def': [], 'base_spe': [], 'base_spc': []
          }
name_search_string = r'\)">([A-Z][^<]*)<'
stat_search_string = r'">(\d+)$'

def get_name(line):
    name = ''
    match = re.search(name_search_string, line)
    if match:
        name = match.group(1)
    else:
        print('Error in get_name: '
              f'no match found for line={line}. '
              'Returning empty string')
    return name

def get_stat(line):
    stat = -1
    match = re.search(stat_search_string, line)
    if match:
        stat = int(match.group(1))
    else:
        print('Error in get_stat: '
              f'no match found for line={line}. '
              'Returning -1')
    return stat

def get_names_and_stats():
    i_line = 0
    while True:
        lines = html_lines[i_line:]
        species_line_num = get_matching_line_number(
            name_search_string, lines)
        if species_line_num < 0:
            break
        i_line += species_line_num
        name = get_name(lines[species_line_num])
        print(name)
        bs_dict['name'].append(name)
        bs_dict['base_hp'].append(get_stat(lines[species_line_num+2]))
        bs_dict['base_att'].append(get_stat(lines[species_line_num+4]))
        bs_dict['base_def'].append(get_stat(lines[species_line_num+6]))
        bs_dict['base_spe'].append(get_stat(lines[species_line_num+8]))
        bs_dict['base_spc'].append(get_stat(lines[species_line_num+10]))
        i_line += 11
        if name == 'Mew':
            break

In [97]:
get_names_and_stats()

Bulbasaur
Ivysaur
Venusaur
Charmander
Charmeleon
Charizard
Squirtle
Wartortle
Blastoise
Caterpie
Metapod
Butterfree
Weedle
Kakuna
Beedrill
Pidgey
Pidgeotto
Pidgeot
Rattata
Raticate
Spearow
Fearow
Ekans
Arbok
Pikachu
Raichu
Sandshrew
Sandslash
Nidoran♀
Nidorina
Nidoqueen
Nidoran♂
Nidorino
Nidoking
Clefairy
Clefable
Vulpix
Ninetales
Jigglypuff
Wigglytuff
Zubat
Golbat
Oddish
Gloom
Vileplume
Paras
Parasect
Venonat
Venomoth
Diglett
Dugtrio
Meowth
Persian
Psyduck
Golduck
Mankey
Primeape
Growlithe
Arcanine
Poliwag
Poliwhirl
Poliwrath
Abra
Kadabra
Alakazam
Machop
Machoke
Machamp
Bellsprout
Weepinbell
Victreebel
Tentacool
Tentacruel
Geodude
Graveler
Golem
Ponyta
Rapidash
Slowpoke
Slowbro
Magnemite
Magneton
Farfetch'd
Doduo
Dodrio
Seel
Dewgong
Grimer
Muk
Shellder
Cloyster
Gastly
Haunter
Gengar
Onix
Drowzee
Hypno
Krabby
Kingler
Voltorb
Electrode
Exeggcute
Exeggutor
Cubone
Marowak
Hitmonlee
Hitmonchan
Lickitung
Koffing
Weezing
Rhyhorn
Rhydon
Chansey
Tangela
Kangaskhan
Horsea
Seadra
Goldeen
Seaking

In [98]:
bs_df = pd.DataFrame(data=bs_dict, index=np.array(indices)-1)

In [99]:
# Fix Nidoran names -- html had symbols for F and M
bs_df.loc[28, 'name'] = 'Nidoran F'
bs_df.loc[31, 'name'] = 'Nidoran M'

In [101]:
bs_df.iloc[28]

index              29
name        Nidoran F
base_hp            55
base_att           47
base_def           52
base_spe           41
base_spc           40
Name: 28, dtype: object

In [102]:
# original method using manually copy-pasted csv file
bs_df_orig = pd.read_csv('base_stats.csv')
diffs = (bs_df_orig != bs_df).any(1)
diffs[diffs == True]
# exactly the same!!

ValueError: Can only compare identically-labeled DataFrame objects

## `bs_df` has species index, name, and base stats
Next we want to get type information. We'll pull this from the pokemondb.net stats table

In [103]:
types_url = (
    'https://pokemondb.net/pokedex/stats/gen1'
)

page = urllib.request.urlopen(types_url)
html_bytes = page.read()
html = html_bytes.decode('utf-8')

In [104]:
print(html)

<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="utf-8">
	<title>Generation 1 new Pokémon stats | Pokémon Database</title>

	<link rel="preconnect" href="https://img.pokemondb.net">
	<link rel="preconnect" href="https://s.pokemondb.net">
	<link rel="preload" href="/static/fonts/fira-sans-v17-latin-400.woff2" as="font" type="font/woff2" crossorigin>
	<link rel="preload" href="/static/fonts/fira-sans-v17-latin-400i.woff2" as="font" type="font/woff2" crossorigin>
	<link rel="preload" href="/static/fonts/fira-sans-v17-latin-600.woff2" as="font" type="font/woff2" crossorigin>
		<link rel="stylesheet" href="/static/css/pokemondb-9ad4b87c02.css">

	<meta name="viewport" content="width=device-width, initial-scale=1">

	<meta property="og:description" name="description" content="List of new Pokémon introduced in Gen 1 (Red/Blue/Yellow) along with their stats.">
	<link rel="canonical" href="https://pokemondb.net/pokedex/stats/gen1">
	<meta property="og:url" content="https://pokemondb.net/

In [105]:
html_lines = html.split('\n')

In [106]:
index_list = []
type1_list = []
type2_list = []
starting_line = 0
for i, species in enumerate(bs_df['name']):
    lines = html_lines[starting_line:]
    search_string = '">' + species + '<'
    match = None
    species_line = -1
    line_idx = 0
    while not match and (
        species_line < len(lines) - 1
    ):
        species_line += 1
        match = re.search(search_string, lines[species_line])
    if not match:
        print(f'Hit the end of the file. species={species}, starting_line={starting_line}, species_line={species_line}')
        continue
    the_line = lines[species_line]
    starting_line += species_line
    line_idx = match.span()[1]
    search_string = r'>([A-Z][a-z]+)<'
    match = re.search(search_string, the_line[line_idx:])
    type1 = match.group(1)
    type2 = 'None'
    line_idx += match.span()[1]
    match = re.search(search_string, the_line[line_idx:])
    if match:
        type2 = match.group(1)
    # print(f'Name: {species}\tType 1: {type1}\tType 2: {type2}')
    index_list.append(i+1)
    type1_list.append(type1)
    type2_list.append(type2)

Hit the end of the file. species=Nidoran F, starting_line=585, species_line=1624
Hit the end of the file. species=Nidoran M, starting_line=624, species_line=1585


Nidoran F and Nidoran M give us trouble because of the symbols in the name. Need to manually add these to the lists.

In [129]:
for i, t in enumerate(type1_list):
    print(f'i={i}, type1={t}')

i=0, type1=Grass
i=1, type1=Grass
i=2, type1=Grass
i=3, type1=Fire
i=4, type1=Fire
i=5, type1=Fire
i=6, type1=Water
i=7, type1=Water
i=8, type1=Water
i=9, type1=Bug
i=10, type1=Bug
i=11, type1=Bug
i=12, type1=Bug
i=13, type1=Bug
i=14, type1=Bug
i=15, type1=Normal
i=16, type1=Normal
i=17, type1=Normal
i=18, type1=Normal
i=19, type1=Normal
i=20, type1=Normal
i=21, type1=Normal
i=22, type1=Poison
i=23, type1=Poison
i=24, type1=Electric
i=25, type1=Electric
i=26, type1=Ground
i=27, type1=Ground
i=28, type1=Poison
i=29, type1=Poison
i=30, type1=Poison
i=31, type1=Poison
i=32, type1=Poison
i=33, type1=Poison
i=34, type1=Fairy
i=35, type1=Fairy
i=36, type1=Fire
i=37, type1=Fire
i=38, type1=Normal
i=39, type1=Normal
i=40, type1=Poison
i=41, type1=Poison
i=42, type1=Grass
i=43, type1=Grass
i=44, type1=Grass
i=45, type1=Bug
i=46, type1=Bug
i=47, type1=Bug
i=48, type1=Bug
i=49, type1=Ground
i=50, type1=Ground
i=51, type1=Normal
i=52, type1=Normal
i=53, type1=Water
i=54, type1=Water
i=55, type1=Fi

In [130]:
bs_df.iloc[34]

index             35
name        Clefairy
base_hp           70
base_att          45
base_def          48
base_spe          35
base_spc          60
Name: 34, dtype: object

In [128]:
index_list.insert(28, 29)
index_list.insert(31, 32)
type1_list.insert(28, 'Poison')
type1_list.insert(31, 'Poison')
type2_list.insert(28, 'None')
type2_list.insert(31, 'None')

In [131]:
series_type1 = pd.Series(data=type1_list)
series_type2 = pd.Series(data=type2_list)

In [132]:
bst_df = bs_df.copy()
bst_df['type1'] = series_type1
bst_df['type2'] = series_type2

In [133]:
bst_df

,index,name,base_hp,base_att,base_def,base_spe,base_spc,type1,type2
0,1,Bulbasaur,45,49,49,45,65,Grass,Poison
1,2,Ivysaur,60,62,63,60,80,Grass,Poison
2,3,Venusaur,80,82,83,80,100,Grass,Poison
3,4,Charmander,39,52,43,65,50,Fire,None
4,5,Charmeleon,58,64,58,80,65,Fire,None
...,...,...,...,...,...,...,...,...,...
146,147,Dratini,41,64,45,50,50,Dragon,None
147,148,Dragonair,61,84,65,70,70,Dragon,None
148,149,Dragonite,91,134,95,80,100,Dragon,Flying
149,150,Mewtwo,106,110,90,130,154,Psychic,None


## `bst_df` now contains base stats and type info
Need to add randomizer levels and move lists. Randomizer levels are listed in `Gen1RandomizerLevels.csv`.

In [134]:
rl_df = pd.read_csv('Gen1RandomizerLevels.csv')
rl_df

,name,randomizer_level
0,Abra,84
1,Aerodactyl,75
2,Alakazam,68
3,Arbok,78
4,Arcanine,75
...,...,...
146,Weepinbell,80
147,Weezing,76
148,Wigglytuff,76
149,Zapdos,68


In [135]:
bstr_df = bst_df.merge(rl_df, on='name', how='inner')
bstr_df

,index,name,base_hp,base_att,base_def,base_spe,base_spc,type1,type2,randomizer_level
0,1,Bulbasaur,45,49,49,45,65,Grass,Poison,89
1,2,Ivysaur,60,62,63,60,80,Grass,Poison,80
2,3,Venusaur,80,82,83,80,100,Grass,Poison,74
3,4,Charmander,39,52,43,65,50,Fire,None,90
4,5,Charmeleon,58,64,58,80,65,Fire,None,81
...,...,...,...,...,...,...,...,...,...,...
146,147,Dratini,41,64,45,50,50,Dragon,None,89
147,148,Dragonair,61,84,65,70,70,Dragon,None,80
148,149,Dragonite,91,134,95,80,100,Dragon,Flying,74
149,150,Mewtwo,106,110,90,130,154,Psychic,None,60


## `bstr_df` now contains stats, types, and randomizer level
Last we need to include movepools. Define functions to grab this from either Bulbapedia or PokemonDB.

In [136]:
def get_html_lines(species):
    name = species.lower()
    if name == 'mr. mime':
        name = 'mr-mime'
    if name == 'nidoran f':
        name = 'nidoran-f'
    if name == 'nidoran m':
        name = 'nidoran-m'
    if name == "farfetch'd":
        name = 'farfetchd'
    url = 'https://pokemondb.net/pokedex/' + name + '/moves/1'
    print(f'Opening url for {species}')
    page = urllib.request.urlopen(url)
    print(f'Got page for {species}')
    html_bytes = page.read()
    html = html_bytes.decode('utf-8')
    html_lines = html.split('\n')
    return html_lines

In [137]:
tauros_lines = get_html_lines('Tauros')
print(tauros_lines[211])

Opening url for Tauros
Got page for Tauros
			<div class="grid-row"> <div class="grid-col span-lg-6"><h3>Moves learnt by level up</h3> <p class="text-small"><em>Tauros</em> learns the following moves in Pokémon Red &amp; Blue at the levels specified.</p> <div class="resp-scroll"><table class="data-table"><thead><tr><th class="sorting" data-sort-type="int"><div class="sortwrap">Lv.</div></th> <th class="sorting" data-sort-type="string"><div class="sortwrap">Move</div></th> <th class="sorting" data-sort-type="string"><div class="sortwrap">Type</div></th> <th class="sorting" data-sort-type="string"><div class="sortwrap">Cat.</div></th> <th class="sorting" data-sort-type="int" data-sort-default="desc" data-blanks="1"><div class="sortwrap">Power</div></th> <th class="sorting" data-sort-type="int" data-sort-default="desc" data-blanks="1"><div class="sortwrap">Acc.</div></th> </tr></thead><tbody><tr><td class="cell-num">1</td><td class="cell-name"><a class="ent-name" href="/move/tackle" title

In [138]:
def get_matching_line_number(pattern, lines):
    has_match = False
    for i, line in enumerate(lines):
        match = re.search(pattern, line)
        if match:
            has_match = True
            break
    return i if has_match else -1

In [139]:
get_matching_line_number(r'Lv\.', tauros_lines)

211

In [140]:
def get_movepool(lines):
    moves = []
    starting_line = get_matching_line_number(r'Lv\.', lines)
    pat = r'"View details for ([A-Z][^"]*)"'
    for line in lines[starting_line:]:
        match = re.search(pat, line)
        if match:
            moves.append(match.group(1))
    return set(moves)

In [141]:
tauros_moves = get_movepool(tauros_lines)
print(tauros_moves)

{'Thunderbolt', 'Mimic', 'Take Down', 'Rage', 'Leer', 'Double Team', 'Ice Beam', 'Body Slam', 'Stomp', 'Fire Blast', 'Substitute', 'Hyper Beam', 'Tail Whip', 'Blizzard', 'Skull Bash', 'Bide', 'Strength', 'Toxic', 'Earthquake', 'Fissure', 'Thunder', 'Double-Edge', 'Tackle', 'Horn Drill', 'Rest'}


In [142]:
def convert_set(set):
    ret = ''
    sorted_set = sorted(set)
    for element in sorted_set:
        ret += element
        ret += ';'
    return ret[:-1]

In [143]:
def get_movepool_series():
    species_moves = []
    for species in bstr_df['name']:
        lines = get_html_lines(species)
        move_set = get_movepool(lines)
        print(f'{species}: {len(move_set)}')
        move_str = convert_set(move_set)
        species_moves.append(move_str)
    move_series = pd.Series(data=species_moves)
    return move_series

In [147]:
move_series = get_movepool_series()

Opening url for Bulbasaur
Got page for Bulbasaur
Bulbasaur: 23
Opening url for Ivysaur
Got page for Ivysaur
Ivysaur: 23
Opening url for Venusaur
Got page for Venusaur
Venusaur: 24
Opening url for Charmander
Got page for Charmander
Charmander: 31
Opening url for Charmeleon
Got page for Charmeleon
Charmeleon: 31
Opening url for Charizard
Got page for Charizard
Charizard: 35
Opening url for Squirtle
Got page for Squirtle
Squirtle: 30
Opening url for Wartortle
Got page for Wartortle
Wartortle: 30
Opening url for Blastoise
Got page for Blastoise
Blastoise: 33
Opening url for Caterpie
Got page for Caterpie
Caterpie: 2
Opening url for Metapod
Got page for Metapod
Metapod: 3
Opening url for Butterfree
Got page for Butterfree
Butterfree: 30
Opening url for Weedle
Got page for Weedle
Weedle: 2
Opening url for Kakuna
Got page for Kakuna
Kakuna: 3
Opening url for Beedrill
Got page for Beedrill
Beedrill: 24
Opening url for Pidgey
Got page for Pidgey
Pidgey: 21
Opening url for Pidgeotto
Got page for

Got page for Aerodactyl
Aerodactyl: 22
Opening url for Snorlax
Got page for Snorlax
Snorlax: 39
Opening url for Articuno
Got page for Articuno
Articuno: 23
Opening url for Zapdos
Got page for Zapdos
Zapdos: 24
Opening url for Moltres
Got page for Moltres
Moltres: 21
Opening url for Dratini
Got page for Dratini
Dratini: 28
Opening url for Dragonair
Got page for Dragonair
Dragonair: 29
Opening url for Dragonite
Got page for Dragonite
Dragonite: 31
Opening url for Mewtwo
Got page for Mewtwo
Mewtwo: 43
Opening url for Mew
Got page for Mew
Mew: 57


In [148]:
df_all = bstr_df.copy()
df_all['movepool'] = move_series

In [149]:
df_all

,index,name,base_hp,base_att,base_def,base_spe,base_spc,type1,type2,randomizer_level,movepool
0,1,Bulbasaur,45,49,49,45,65,Grass,Poison,89,Bide;Body Slam;Cut;Double Team;Double-Edge;Gro...
1,2,Ivysaur,60,62,63,60,80,Grass,Poison,80,Bide;Body Slam;Cut;Double Team;Double-Edge;Gro...
2,3,Venusaur,80,82,83,80,100,Grass,Poison,74,Bide;Body Slam;Cut;Double Team;Double-Edge;Gro...
3,4,Charmander,39,52,43,65,50,Fire,None,90,Bide;Body Slam;Counter;Cut;Dig;Double Team;Dou...
4,5,Charmeleon,58,64,58,80,65,Fire,None,81,Bide;Body Slam;Counter;Cut;Dig;Double Team;Dou...
...,...,...,...,...,...,...,...,...,...,...,...
146,147,Dratini,41,64,45,50,50,Dragon,None,89,Agility;Bide;Blizzard;Body Slam;BubbleBeam;Dou...
147,148,Dragonair,61,84,65,70,70,Dragon,None,80,Agility;Bide;Blizzard;Body Slam;BubbleBeam;Dou...
148,149,Dragonite,91,134,95,80,100,Dragon,Flying,74,Agility;Bide;Blizzard;Body Slam;BubbleBeam;Dou...
149,150,Mewtwo,106,110,90,130,154,Psychic,None,60,Amnesia;Barrier;Bide;Blizzard;Body Slam;Bubble...


## One more problem to fix: fairy type
The PokemonDB info for types includes Fairy for a few pokemon. Need to remove this

In [150]:
# change primary type to Normal
df_all.loc[df_all['type1'] == 'Fairy', 'type1'] = 'Normal'

# change secondary type to None
df_all.loc[df_all['type2'] == 'Fairy', 'type2'] = 'None'

## `df_all` now contains everything!
Save it to a file

In [151]:
df_all.to_csv('species_list.csv', index=False)

In [152]:
# testing how to convert a row to a dict, for Pokemon.__init__()
df_all[df_all['name'] == 'Tauros'].to_dict(orient='records')[0]

{'index': 128,
 'name': 'Tauros',
 'base_hp': 75,
 'base_att': 100,
 'base_def': 95,
 'base_spe': 110,
 'base_spc': 70,
 'type1': 'Normal',
 'type2': 'None',
 'randomizer_level': 68,
 'movepool': 'Bide;Blizzard;Body Slam;Double Team;Double-Edge;Earthquake;Fire Blast;Fissure;Horn Drill;Hyper Beam;Ice Beam;Leer;Mimic;Rage;Rest;Skull Bash;Stomp;Strength;Substitute;Tackle;Tail Whip;Take Down;Thunder;Thunderbolt;Toxic'}

In [3]:
df_all = pd.read_csv('species_list.csv')

In [4]:
df_all

,index,name,base_hp,base_att,base_def,base_spe,base_spc,type1,type2,randomizer_level,movepool
0,1,Bulbasaur,45,49,49,45,65,Grass,Poison,89,Bide;Body Slam;Cut;Double Team;Double-Edge;Gro...
1,2,Ivysaur,60,62,63,60,80,Grass,Poison,80,Bide;Body Slam;Cut;Double Team;Double-Edge;Gro...
2,3,Venusaur,80,82,83,80,100,Grass,Poison,74,Bide;Body Slam;Cut;Double Team;Double-Edge;Gro...
3,4,Charmander,39,52,43,65,50,Fire,None,90,Bide;Body Slam;Counter;Cut;Dig;Double Team;Dou...
4,5,Charmeleon,58,64,58,80,65,Fire,None,81,Bide;Body Slam;Counter;Cut;Dig;Double Team;Dou...
...,...,...,...,...,...,...,...,...,...,...,...
146,147,Dratini,41,64,45,50,50,Dragon,None,89,Agility;Bide;Blizzard;Body Slam;BubbleBeam;Dou...
147,148,Dragonair,61,84,65,70,70,Dragon,None,80,Agility;Bide;Blizzard;Body Slam;BubbleBeam;Dou...
148,149,Dragonite,91,134,95,80,100,Dragon,Flying,74,Agility;Bide;Blizzard;Body Slam;BubbleBeam;Dou...
149,150,Mewtwo,106,110,90,130,154,Psychic,None,60,Amnesia;Barrier;Bide;Blizzard;Body Slam;Bubble...


# Moves list

In [9]:
moves_url = 'https://pokemondb.net/move/generation/1'
page = urllib.request.urlopen(moves_url)
html_bytes = page.read()
html = html_bytes.decode('utf-8')
html_lines = html.split('\n')

In [11]:
for i, line in enumerate(html_lines):
    print(f'{i})', line)

0) <!DOCTYPE html>
1) <html lang="en">
2) <head>
3) 	<meta charset="utf-8">
4) 	<title>Pokémon moves from Generation 1 | Pokémon Database</title>
5) 
6) 	<link rel="preconnect" href="https://img.pokemondb.net">
7) 	<link rel="preconnect" href="https://s.pokemondb.net">
8) 	<link rel="preload" href="/static/fonts/fira-sans-v17-latin-400.woff2" as="font" type="font/woff2" crossorigin>
9) 	<link rel="preload" href="/static/fonts/fira-sans-v17-latin-400i.woff2" as="font" type="font/woff2" crossorigin>
10) 	<link rel="preload" href="/static/fonts/fira-sans-v17-latin-600.woff2" as="font" type="font/woff2" crossorigin>
11) 		<link rel="stylesheet" href="/static/css/pokemondb-597e7dc95a.css">
12) 
13) 	<meta name="viewport" content="width=device-width, initial-scale=1">
14) 
15) 	<meta property="og:description" name="description" content="Complete list of Pokémon attacks introduced in Generation 1 (Red, Blue, Yellow).">
16) 	<link rel="canonical" href="https://pokemondb.net/move/generation/1">

In [207]:
name_search_string = r'View details[^>]*>([A-Z][^<]*)<'
type_search_string = r'href="/type/\w*" >([A-Z][^<]*)<'
cat_search_string = r'alt="([A-Z][^"]*)"'
num_search_string = r'>(\d+|—)<'
lines_between_moves = 4

moves_dict = {'index': [], 'name': [], 'web_category': [], 'type': [],
              'max_pp': [], 'base_power': [], 'accuracy': [],
             }

def get_match(search_string, line):
    item = ''
    match = re.search(search_string, line)
    if match:
        item = match.group(1)
    else:
        print('Error in get_match: '
              f'no match found for line={line}. '
              'Returning empty string')
    return item

def get_moves_and_data():
    index = 0
    i_line = 0
    while True:
        lines = html_lines[i_line:]
        move_line_num = get_matching_line_number(
            name_search_string, lines)
        if move_line_num < 0:
            break
        index += 1
        i_line += move_line_num
        the_line = lines[move_line_num]
        name = get_match(name_search_string, the_line)
        print(name)
        moves_dict['index'].append(index)
        moves_dict['name'].append(name)
        moves_dict['web_category'].append(get_match(cat_search_string, the_line))
        moves_dict['type'].append(get_match(type_search_string, the_line))
        nums = re.findall(num_search_string, the_line)
        # translate '—' to '-' and convert numeric strings to ints
        formatted_nums = []
        for num in nums:
            try:
                formatted_nums.append(int(num))
            except ValueError:
                formatted_nums.append('-')
        if len(nums) == 3:
            moves_dict['base_power'].append(formatted_nums[0])
            moves_dict['accuracy'].append(formatted_nums[1])
            moves_dict['max_pp'].append(formatted_nums[2])
        else:
            print(f'Found weird nums list: {nums}')
        if name == 'Swift':
            moves_dict['base_power'].append(60)
            moves_dict['accuracy'].append('-')
            moves_dict['max_pp'].append(20)
        i_line += lines_between_moves
        if name == 'Wrap':
            break

In [208]:
get_moves_and_data()
for key in moves_dict.keys():
    print(f'{key}: {len(moves_dict[key])}')
moves_df = pd.DataFrame(moves_dict)

Absorb
Acid
Acid Armor
Agility
Amnesia
Aurora Beam
Barrage
Barrier
Bide
Bind
Bite
Blizzard
Body Slam
Bone Club
Bonemerang
Bubble
Bubble Beam
Clamp
Comet Punch
Confuse Ray
Confusion
Constrict
Conversion
Counter
Crabhammer
Cut
Defense Curl
Dig
Disable
Dizzy Punch
Double Kick
Double Slap
Double Team
Double-Edge
Dragon Rage
Dream Eater
Drill Peck
Earthquake
Egg Bomb
Ember
Explosion
Fire Blast
Fire Punch
Fire Spin
Fissure
Flamethrower
Flash
Fly
Focus Energy
Fury Attack
Fury Swipes
Glare
Growl
Growth
Guillotine
Gust
Harden
Haze
Headbutt
High Jump Kick
Horn Attack
Horn Drill
Hydro Pump
Hyper Beam
Hyper Fang
Hypnosis
Ice Beam
Ice Punch
Jump Kick
Karate Chop
Kinesis
Leech Life
Leech Seed
Leer
Lick
Light Screen
Lovely Kiss
Low Kick
Meditate
Mega Drain
Mega Kick
Mega Punch
Metronome
Mimic
Minimize
Mirror Move
Mist
Night Shade
Pay Day
Peck
Petal Dance
Pin Missile
Poison Gas
Poison Powder
Poison Sting
Pound
Psybeam
Psychic
Psywave
Quick Attack
Rage
Razor Leaf
Razor Wind
Recover
Reflect
Rest
Roar
Ro

`moves_df` now has our basic info. But we need to refine this quite a bit:
1. Remove all the moves we don't want
2. Some moves are listed by their later-gen names -- e.g. 'ViceGrip' --> 'Vise Grip'. Need to fix these
3. One move (Bite) is given its later-gen type (Normal --> Dark)
4. Some moves are given their later-gen damage category, i.e. Physical/Special. Need to fix this.
5. Change category where applicable -- right now all non-damaging moves are 'Status' category
6. Some moves are given later-gen base power or accuracy -- e.g. Double-Edge base power 100 --> 120. Need to fix these
7. Add correct values for stat/status accuracy, stat change, etc. for all the moves we do want to implement
8. Add "unique_effect" column for unique moves -- e.g. recoil, multihit, OHKO etc.


Some additional notes on moves that require special implementation:

- definitely remove: ['Bide', 'Focus Energy', 'Haze', 'Mist', 'Rage', 'Razor Wind', 'Roar', 'Skull Bash', 'Sky Attack', 'Teleport', 'Whirlwind']

- 10% flinch: [Bite', 'Bone Club', 'Hyper Fang']
- 30% flinch: ['Headbutt', 'Low Kick', 'Stomp']

- recoil 25%: ['Double-Edge', 'Submission', 'Take Down']
- recoil 50%: ['Struggle']
- crash (1 HP): ['Hi Jump Kick', 'Jump Kick']

- multihit random: ['Barrage', 'Comet Punch', 'DoubleSlap', 'Fury Attack', 'Fury Swipes', 'Pin Missile', 'Spike Cannon']
- multihit 2x: ['Bonemerang', 'Double Kick', 'Twineedle']

- hp drain: ['Absorb', 'Dream Eater', 'Leech Life', 'Mega Drain']
- recovery: ['Recover', 'Softboiled']

- increased crit: ['Crabhammer', 'Karate Chop', 'Razor Leaf', 'Slash']

- fixed damage: ['Dragon Rage', 'Night Shade', 'Psywave', 'Seismic Toss', 'SonicBoom', 'Super Fang']

- OHKO: ['Fissure', 'Guillotine', 'Horn Drill']

- wrapping: ['Bind', 'Clamp', 'Fire Spin', 'Wrap']

- other unique: ['Counter', 'Dig', 'Explosion', 'Fly', 'Hyper Beam', 'Leech Seed', 'Light Screen', 'Quick Attack', 'Reflect', 'Selfdestruct', 'SolarBeam', 'Swift', 'Thrash', 'Toxic']

- implement later: 'Conversion', 'Disable', 'Metronome', 'Mimic', 'Mirror Move', 'Substitute', 'Transform', 

In [209]:
# 1. Remove moves we definitely don't want
to_remove = ['Bide', 'Focus Energy', 'Haze', 'Mist', 'Rage', 'Razor Wind', 
          'Skull Bash', 'Sky Attack', 'Teleport']

def remove(names, df):
    copy = df.copy()
    for name in names:
        idx = copy[copy['name'] == name].index
        copy = copy.drop(idx)
    return copy

moves_df = remove(to_remove, moves_df)

In [210]:
# 2. Correct move names
wrong_names = ['Bubble Beam', 'Double Slap', 'High Jump Kick', 'Poison Powder', 'Sand Attack', 'Self-Destruct', 
               'Smokescreen', 'Soft-Boiled', 'Solar Beam', 'Sonic Boom', 'Thunder Punch', 'Thunder Shock', 
               'Vise Grip']
right_names = ['BubbleBeam', 'DoubleSlap', 'Hi Jump Kick', 'PoisonPowder', 'Sand-Attack', 'Selfdestruct', 
               'SmokeScreen', 'Softboiled', 'SolarBeam', 'SonicBoom', 'ThunderPunch', 'ThunderShock', 
               'ViceGrip']
names = zip(wrong_names, right_names)
for pair in names:
    moves_df.loc[moves_df['name'] == pair[0], 'name'] = pair[1]

print(moves_df['name'].values)

['Absorb' 'Acid' 'Acid Armor' 'Agility' 'Amnesia' 'Aurora Beam' 'Barrage'
 'Barrier' 'Bind' 'Bite' 'Blizzard' 'Body Slam' 'Bone Club' 'Bonemerang'
 'Bubble' 'BubbleBeam' 'Clamp' 'Comet Punch' 'Confuse Ray' 'Confusion'
 'Constrict' 'Conversion' 'Counter' 'Crabhammer' 'Cut' 'Defense Curl'
 'Dig' 'Disable' 'Dizzy Punch' 'Double Kick' 'DoubleSlap' 'Double Team'
 'Double-Edge' 'Dragon Rage' 'Dream Eater' 'Drill Peck' 'Earthquake'
 'Egg Bomb' 'Ember' 'Explosion' 'Fire Blast' 'Fire Punch' 'Fire Spin'
 'Fissure' 'Flamethrower' 'Flash' 'Fly' 'Fury Attack' 'Fury Swipes'
 'Glare' 'Growl' 'Growth' 'Guillotine' 'Gust' 'Harden' 'Headbutt'
 'Hi Jump Kick' 'Horn Attack' 'Horn Drill' 'Hydro Pump' 'Hyper Beam'
 'Hyper Fang' 'Hypnosis' 'Ice Beam' 'Ice Punch' 'Jump Kick' 'Karate Chop'
 'Kinesis' 'Leech Life' 'Leech Seed' 'Leer' 'Lick' 'Light Screen'
 'Lovely Kiss' 'Low Kick' 'Meditate' 'Mega Drain' 'Mega Kick' 'Mega Punch'
 'Metronome' 'Mimic' 'Minimize' 'Mirror Move' 'Night Shade' 'Pay Day'
 'Peck' 'Peta

In [211]:
# 3. Correct Bite's type
moves_df.loc[moves_df['name'] == 'Bite', 'type'] = 'Normal'

# 4. Correct future-gen damage categories
types = ['None', 'Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock',
         'Bug', 'Ghost', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic',
         'Ice', 'Dragon']

def get_category(row):
    if row['web_category'] == 'Status':
        cat = 'Status'
    else:
        cat = 'Physical' if types.index(row['type']) < 9 else 'Special'
    return cat

moves_df['category'] = moves_df.apply(get_category, axis=1)

In [212]:
# 5. Change "Status" category to separate "Stat", "Status", and "Unique" categories
# First, list these moves
web_status = moves_df['name'][moves_df['category'] == 'Status'].values
print('All "Status" moves:', web_status)

stat_moves = ['Acid Armor', 'Agility', 'Amnesia', 'Barrier', 'Defense Curl', 'Double Team',
              'Flash', 'Growl', 'Growth', 'Harden', 'Kinesis', 'Leer', 'Meditate', 'Minimize',
              'Sand Attack', 'Screech', 'Sharpen', 'Smokescreen', 'String Shot', 'Swords Dance',
              'Tail Whip', 'Withdraw']
print('Stat-changing moves excluded:', sorted(set(web_status) - set(stat_moves)))

status_moves = ['Confuse Ray', 'Glare', 'Hypnosis', 'Lovely Kiss', 'Poison Gas', 'PoisonPowder',
                'Rest', 'Sing', 'Sleep Powder', 'Spore', 'Stun Spore', 'Supersonic', 'Thunder Wave']
unique_moves = sorted(set(web_status) - set(stat_moves) - set(status_moves))
print('Remaining unique moves:', unique_moves)

All "Status" moves: ['Acid Armor' 'Agility' 'Amnesia' 'Barrier' 'Confuse Ray' 'Conversion'
 'Defense Curl' 'Disable' 'Double Team' 'Flash' 'Glare' 'Growl' 'Growth'
 'Harden' 'Hypnosis' 'Kinesis' 'Leech Seed' 'Leer' 'Light Screen'
 'Lovely Kiss' 'Meditate' 'Metronome' 'Mimic' 'Minimize' 'Mirror Move'
 'Poison Gas' 'PoisonPowder' 'Recover' 'Reflect' 'Rest' 'Roar'
 'Sand-Attack' 'Screech' 'Sharpen' 'Sing' 'Sleep Powder' 'SmokeScreen'
 'Softboiled' 'Splash' 'Spore' 'String Shot' 'Stun Spore' 'Substitute'
 'Supersonic' 'Swords Dance' 'Tail Whip' 'Thunder Wave' 'Toxic'
 'Transform' 'Whirlwind' 'Withdraw']
Stat-changing moves excluded: ['Confuse Ray', 'Conversion', 'Disable', 'Glare', 'Hypnosis', 'Leech Seed', 'Light Screen', 'Lovely Kiss', 'Metronome', 'Mimic', 'Mirror Move', 'Poison Gas', 'PoisonPowder', 'Recover', 'Reflect', 'Rest', 'Roar', 'Sand-Attack', 'Sing', 'Sleep Powder', 'SmokeScreen', 'Softboiled', 'Splash', 'Spore', 'Stun Spore', 'Substitute', 'Supersonic', 'Thunder Wave', 'Toxic

In [213]:
# Change the category for these moves
def change_cat(df, names, new_cat):
    copy = df.copy()
    for name in names:
        idx = copy[copy['name'] == name].index
        copy.loc[idx, 'category'] = new_cat
    return copy

moves_df = change_cat(moves_df, stat_moves, 'Stat')
moves_df = change_cat(moves_df, unique_moves, 'Unique')

Need to change a bunch of additional damaging moves with unique effects -- will come back to this later

In [215]:
# 6. Fix moves with future-gen base power or accuracy
# Can get this from the gen 1 learnset page for each species, similar to how we got the movepools

pow_dict = {}
acc_dict = {}
pp_dict = {}
move_names = set()
name_search_string = r'title="([A-Z][^(]*) \(move\)"'
num_search_string = r'>(\d+|—)%?$'

def get_html_lines(species):
    name = str(species)
    if name == 'Mr. Mime':
        name = 'Mr._Mime'
    if name == "Farfetch'd":
        name = 'Farfetch%27d'
    # Nidoran gives us too much trouble -- just skip it entirely
    # (all moves of both Nidorans are covered by other Pokemon)
    if name in ['Nidoran F', 'Nidoran M']:
        return []
    url = 'https://pokemondb.net/pokedex/' + name + '/moves/1'
    url = 'https://bulbapedia.bulbagarden.net/wiki/' + name + '_(Pok%C3%A9mon)/Generation_I_learnset'
    print(f'Opening url for {species}: url="{url}"')
    page = urllib.request.urlopen(url)
    print(f'Got page for {species}')
    html_bytes = page.read()
    html = html_bytes.decode('utf-8')
    html_lines = html.split('\n')
    return html_lines

def get_matching_line_number(pattern, lines):
    has_match = False
    for i, line in enumerate(lines):
        match = re.search(pattern, line)
        if match:
            has_match = True
            break
    return i if has_match else -1

def get_match(pattern, line):
    ret = None
    match = re.search(pattern, line)
    if match:
        val = match.group(1)
        try:
            ret = int(val)
        except ValueError:
            ret = '-'
    else:
        print(f'In get_match: no match found in the following line\n\t{line}')
    return ret

def get_move_name_pow_acc_pp(lines):
    starting_line = get_matching_line_number(r'title="PP"', lines)
    for i, line in enumerate(lines[starting_line:]):
        name_match = re.search(name_search_string, line)
        if name_match:
            name = name_match.group(1)
            if name not in move_names:
                print(f'Found new move: {name}')
                move_names.add(name)
                pow_line = lines[starting_line+i+4]
                power = get_match(num_search_string, pow_line)
                pow_dict[name] = power
                acc_line = lines[starting_line+i+6]
                acc = get_match(num_search_string, acc_line)
                acc_dict[name] = acc
                pp_line = lines[starting_line+i+8]
                pp = get_match(num_search_string, pp_line)
                pp_dict[name] = pp

def get_all_moves_pow_acc_pp():
    pow_dict.clear()
    acc_dict.clear()
    pp_dict.clear()
    move_names.clear()
    for species in df_all['name']:
        # print(f'{species}:')
        lines = get_html_lines(species)
        if not lines:
            continue
        get_move_name_pow_acc_pp(lines)

In [216]:
get_all_moves_pow_acc_pp()
print(f'Found {len(move_names)} moves!')
print(f'power dict has {len(pow_dict.keys())} keys')
print(f'accuracy dict has {len(acc_dict.keys())} keys')
print(f'pp dict has {len(pp_dict.keys())} keys')

Opening url for Bulbasaur: url="https://bulbapedia.bulbagarden.net/wiki/Bulbasaur_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Bulbasaur
Found new move: Tackle
Found new move: Growl
Found new move: Leech Seed
Found new move: Vine Whip
Found new move: PoisonPowder
Found new move: Razor Leaf
Found new move: Growth
Found new move: Sleep Powder
Found new move: SolarBeam
Found new move: Swords Dance
Found new move: Toxic
Found new move: Body Slam
Found new move: Take Down
Found new move: Double-Edge
Found new move: Rage
Found new move: Mega Drain
Found new move: Mimic
Found new move: Double Team
Found new move: Reflect
Found new move: Bide
Found new move: Rest
Found new move: Substitute
Found new move: Cut
Found new move: Defense Curl
Found new move: Flash
Found new move: Headbutt
Found new move: Light Screen
Found new move: Petal Dance
Found new move: Razor Wind
Found new move: Skull Bash
Opening url for Ivysaur: url="https://bulbapedia.bulbagarden.net/wiki/Ivysaur_(Pok%C3%A9mon)/Gen

Got page for Wigglytuff
Opening url for Zubat: url="https://bulbapedia.bulbagarden.net/wiki/Zubat_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Zubat
Found new move: Leech Life
Opening url for Golbat: url="https://bulbapedia.bulbagarden.net/wiki/Golbat_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Golbat
Opening url for Oddish: url="https://bulbapedia.bulbagarden.net/wiki/Oddish_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Oddish
Found new move: Absorb
Opening url for Gloom: url="https://bulbapedia.bulbagarden.net/wiki/Gloom_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Gloom
Opening url for Vileplume: url="https://bulbapedia.bulbagarden.net/wiki/Vileplume_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Vileplume
Opening url for Paras: url="https://bulbapedia.bulbagarden.net/wiki/Paras_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Paras
Found new move: Spore
Opening url for Parasect: url="https://bulbapedia.bulbagarden.net/wiki/Parasect_(Pok%C3%A9mon)/Gener

Got page for Drowzee
Opening url for Hypno: url="https://bulbapedia.bulbagarden.net/wiki/Hypno_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Hypno
Opening url for Krabby: url="https://bulbapedia.bulbagarden.net/wiki/Krabby_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Krabby
Found new move: ViceGrip
Found new move: Guillotine
Found new move: Crabhammer
Opening url for Kingler: url="https://bulbapedia.bulbagarden.net/wiki/Kingler_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Kingler
Opening url for Voltorb: url="https://bulbapedia.bulbagarden.net/wiki/Voltorb_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Voltorb
Opening url for Electrode: url="https://bulbapedia.bulbagarden.net/wiki/Electrode_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Electrode
Opening url for Exeggcute: url="https://bulbapedia.bulbagarden.net/wiki/Exeggcute_(Pok%C3%A9mon)/Generation_I_learnset"
Got page for Exeggcute
Found new move: Barrage
Found new move: Egg Bomb
Opening url for Exeggutor:

In [217]:
# Struggle is missing, naturally
move_names.add('Struggle')
move_names_list = sorted(move_names)
pow_dict['Struggle'] = 50
acc_dict['Struggle'] = 100
pp_dict['Struggle'] = 1
print(f'Gen 1 move list:\n{move_names_list}')

# now to aggregate all the info
pow_list = []
acc_list = []
pp_list = []
for i, row in moves_df.iterrows():
    name = row['name']
    if name not in move_names_list:
        print(f'Name {name} from DF not in Gen 1 list!')
    else:
        pow_list.append(pow_dict[name])
        acc_list.append(acc_dict[name])
        pp_list.append(pp_dict[name])

moves_df['g1_base_power'] = pow_list
moves_df['g1_accuracy'] = acc_list
moves_df['g1_max_pp'] = pp_list


Gen 1 move list:
['Absorb', 'Acid', 'Acid Armor', 'Agility', 'Amnesia', 'Aurora Beam', 'Barrage', 'Barrier', 'Bide', 'Bind', 'Bite', 'Blizzard', 'Body Slam', 'Bone Club', 'Bonemerang', 'Bubble', 'BubbleBeam', 'Clamp', 'Comet Punch', 'Confuse Ray', 'Confusion', 'Constrict', 'Conversion', 'Counter', 'Crabhammer', 'Cut', 'Defense Curl', 'Dig', 'Disable', 'Dizzy Punch', 'Double Kick', 'Double Team', 'Double-Edge', 'DoubleSlap', 'Dragon Rage', 'Dream Eater', 'Drill Peck', 'Earthquake', 'Egg Bomb', 'Ember', 'Explosion', 'Fire Blast', 'Fire Punch', 'Fire Spin', 'Fissure', 'Flamethrower', 'Flash', 'Fly', 'Focus Energy', 'Fury Attack', 'Fury Swipes', 'Glare', 'Growl', 'Growth', 'Guillotine', 'Gust', 'Harden', 'Haze', 'Headbutt', 'Hi Jump Kick', 'Horn Attack', 'Horn Drill', 'Hydro Pump', 'Hyper Beam', 'Hyper Fang', 'Hypnosis', 'Ice Beam', 'Ice Punch', 'Jump Kick', 'Karate Chop', 'Kinesis', 'Leech Life', 'Leech Seed', 'Leer', 'Lick', 'Light Screen', 'Lovely Kiss', 'Low Kick', 'Meditate', 'Mega Dr

In [228]:
# Compare Gen 1 values to original values
pow_diff = moves_df.loc[moves_df['base_power'] != moves_df['g1_base_power']][
    ['name', 'base_power', 'g1_base_power']]
print(f'{len(pow_diff)} moves with changed base power:')
print(pow_diff)
print()

acc_diff = moves_df.loc[moves_df['accuracy'] != moves_df['g1_accuracy']][['name', 'accuracy', 'g1_accuracy']]
print(f'{len(acc_diff)} moves with changed accuracy:')
print(acc_diff)
print()

pp_diff = moves_df.loc[moves_df['max_pp'] != moves_df['g1_max_pp']][['name', 'max_pp', 'g1_max_pp']]
print(f'{len(pp_diff)} moves with changed max pp:')
print(pp_diff)


28 moves with changed base power:
             name base_power g1_base_power
11       Blizzard        110           120
15         Bubble         40            20
24     Crabhammer        100            90
27            Dig         80           100
33    Double-Edge        120           100
40      Explosion        250           170
41     Fire Blast        110           120
43      Fire Spin         35            15
45   Flamethrower         90            95
47            Fly         90            70
59   Hi Jump Kick        130            85
62     Hydro Pump        110           120
66       Ice Beam         90            95
68      Jump Kick        100            70
71     Leech Life         80            20
74           Lick         30            20
77       Low Kick          -            50
90    Petal Dance        120            70
91    Pin Missile         25            14
114  Selfdestruct        200           130
123          Smog         30            20
140          Surf   

From here on, we'll use the _correct_ Gen 1 values for base power, accuracy and max PP.

Now we're in a position to address #7 and #8 -- adding all the extra effects and making sure the categories, accuracies etc. are right. We'll do this in the following order:
- primary status moves
- secondary status moves
- primary stat change moves
- secondary stat change moves
- other unique effects
    - recoil
    - crash
    - multihit
    - HP-draining
    - recovery
    - high-crit-rate
    - fixed damage and OHKO
    - wrapping/trapping
    - other
        - counter
        - dig, fly, solarbeam
        - explosion, selfdestruct
        - light screen, reflect
        - leech seed, toxic
        - quick attack
        - swift
        - thrash, petal dance

To account for all these cases, we ultimately want to make a dictionary of all moves, with move names as keys. The corresponding values will be dicts containing all the move details -- name, type, category, max PP, base power, and accuracy will be mandatory; while status effects, stat changes, unique effects etc. will be optional.

In [232]:
moves_dict = {}

def row_to_dict(row):
    d = {}
    d['name'] = row['name']
    d['type'] = row['type']
    d['category'] = row['category']
    d['base_power'] = row['g1_base_power']
    d['accuracy'] = row['g1_accuracy']
    d['max_pp'] = row['g1_max_pp']
    return d

def fill_moves_dict():
    moves_dict.clear()
    for i, row in moves_df.iterrows():
        moves_dict[row['name']] = row_to_dict(row)

fill_moves_dict()
print(moves_dict)

{'Absorb': {'name': 'Absorb', 'type': 'Grass', 'category': 'Special', 'base_power': 20, 'accuracy': 100, 'max_pp': 20}, 'Acid': {'name': 'Acid', 'type': 'Poison', 'category': 'Physical', 'base_power': 40, 'accuracy': 100, 'max_pp': 30}, 'Acid Armor': {'name': 'Acid Armor', 'type': 'Poison', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 40}, 'Agility': {'name': 'Agility', 'type': 'Psychic', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 30}, 'Amnesia': {'name': 'Amnesia', 'type': 'Psychic', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 20}, 'Aurora Beam': {'name': 'Aurora Beam', 'type': 'Ice', 'category': 'Special', 'base_power': 65, 'accuracy': 100, 'max_pp': 20}, 'Barrage': {'name': 'Barrage', 'type': 'Normal', 'category': 'Physical', 'base_power': 15, 'accuracy': 85, 'max_pp': 20}, 'Barrier': {'name': 'Barrier', 'type': 'Psychic', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 30}, 'Bind': {'name': 'Bind', 

In [238]:
# Now define some helper functions

def view_moves(names):
    for name in names:
        print(moves_dict[name])
        print()

def add_item(name, key, val):
    moves_dict[name][key] = val

def add_items_one_val(names, key, val):
    for name in names:
        moves_dict[name][key] = val

def add_items(names, key, vals):
    for i, name in enumerate(names):
        moves_dict[name][key] = vals[i]

def copy_vals(names, orig_key, new_key):
    for name in names:
        moves_dict[name][new_key] = moves_dict[name][orig_key]

In [251]:
# First address status moves
status_moves = ['Confuse Ray', 'Glare', 'Hypnosis', 'Leech Seed', 'Lovely Kiss', 'Poison Gas', 'PoisonPowder',
                'Rest', 'Sing', 'Sleep Powder', 'Spore', 'Stun Spore', 'Supersonic', 'Thunder Wave']
burn = []
confusion = ['Confuse Ray', 'Supersonic']
freeze = []
leech = ['Leech Seed']
paralysis = ['Glare', 'Stun Spore', 'Thunder Wave']
poison = ['Poison Gas', 'PoisonPowder']
sleep = ['Hypnosis', 'Lovely Kiss', 'Sing', 'Sleep Powder', 'Spore']
toxic = ['Toxic']

# Default target for all status moves is the opponent
add_items_one_val(status_moves, 'status_target', 'other')
# Rest is the exception
add_item('Rest', 'status_target', 'self')
# Next add status effects
add_items_one_val(confusion, 'status', 'CNF')
add_items_one_val(leech, 'status', 'LCH')
add_items_one_val(leech, 'category', 'Status')
add_items_one_val(paralysis, 'status', 'PRZ')
add_items_one_val(poison, 'status', 'PSN')
add_items_one_val(sleep, 'status', 'SLP')
add_item('Rest', 'status', 'SLP')
add_items_one_val(toxic, 'status', 'TXC')
# Finally copy main accuracy over the status accuracy
copy_vals(status_moves, 'accuracy', 'status_accuracy')

view_moves(status_moves)

{'name': 'Confuse Ray', 'type': 'Ghost', 'category': 'Status', 'base_power': '-', 'accuracy': 100, 'max_pp': 10, 'status_target': 'other', 'status': 'CNF', 'status_accuracy': 100}

{'name': 'Glare', 'type': 'Normal', 'category': 'Status', 'base_power': '-', 'accuracy': 75, 'max_pp': 30, 'status_target': 'other', 'status': 'PRZ', 'status_accuracy': 75}

{'name': 'Hypnosis', 'type': 'Psychic', 'category': 'Status', 'base_power': '-', 'accuracy': 60, 'max_pp': 20, 'status_target': 'other', 'status': 'SLP', 'status_accuracy': 60}

{'name': 'Leech Seed', 'type': 'Grass', 'category': 'Status', 'base_power': '-', 'accuracy': 90, 'max_pp': 10, 'status_target': 'other', 'status': 'LCH', 'status_accuracy': 90}

{'name': 'Lovely Kiss', 'type': 'Normal', 'category': 'Status', 'base_power': '-', 'accuracy': 75, 'max_pp': 10, 'status_target': 'other', 'status': 'SLP', 'status_accuracy': 75}

{'name': 'Poison Gas', 'type': 'Poison', 'category': 'Status', 'base_power': '-', 'accuracy': 55, 'max_pp': 4

In [244]:
# Next address damaging moves with secondary status effects
secondary_burn_10pct = ['Ember', 'Fire Blast', 'Fire Punch', 'Flamethrower']
secondary_confusion_10pct = ['Confusion', 'Psybeam']
secondary_flinch_10pct = ['Bite', 'Bone Club', 'Hyper Fang']
secondary_flinch_30pct = ['Headbutt', 'Low Kick', 'Stomp']
secondary_freeze_10pct = ['Blizzard', 'Ice Beam', 'Ice Punch']
secondary_paralysis_10pct = ['Thunder', 'ThunderPunch', 'ThunderShock', 'Thunderbolt']
secondary_paralysis_30pct = ['Body Slam', 'Lick']
secondary_poison_20pct = ['Poison Sting', 'Twineedle']
secondary_poison_40pct = ['Sludge', 'Smog']

secondary_status_moves = (secondary_burn_10pct + secondary_confusion_10pct + secondary_flinch_10pct +
                          secondary_flinch_30pct + secondary_freeze_10pct + secondary_paralysis_10pct +
                          secondary_paralysis_30pct + secondary_poison_20pct + secondary_poison_40pct)
n1 = len(secondary_status_moves)
statuses = ['BRN']*4 + ['CNF']*2 + ['FLN']*6 + ['FRZ']*3 + ['PRZ']*6 + ['PSN']*4
n2 = len(statuses)
status_accs = [10]*9 + [30]*3 + [10]*7 + [30]*2 + [20]*2 + [40]*2
n3 = len(status_accs)

if n1 == n2 and n1 == n3:
    add_items_one_val(secondary_status_moves, 'status_target', 'other')
    add_items(secondary_status_moves, 'status', statuses)
    add_items(secondary_status_moves, 'status_accuracy', status_accs)

view_moves(secondary_status_moves)

{'name': 'Ember', 'type': 'Fire', 'category': 'Special', 'base_power': 40, 'accuracy': 100, 'max_pp': 25, 'status_target': 'other', 'status': 'BRN', 'status_accuracy': 10}

{'name': 'Fire Blast', 'type': 'Fire', 'category': 'Special', 'base_power': 120, 'accuracy': 85, 'max_pp': 5, 'status_target': 'other', 'status': 'BRN', 'status_accuracy': 10}

{'name': 'Fire Punch', 'type': 'Fire', 'category': 'Special', 'base_power': 75, 'accuracy': 100, 'max_pp': 15, 'status_target': 'other', 'status': 'BRN', 'status_accuracy': 10}

{'name': 'Flamethrower', 'type': 'Fire', 'category': 'Special', 'base_power': 95, 'accuracy': 100, 'max_pp': 15, 'status_target': 'other', 'status': 'BRN', 'status_accuracy': 10}

{'name': 'Confusion', 'type': 'Psychic', 'category': 'Special', 'base_power': 50, 'accuracy': 100, 'max_pp': 25, 'status_target': 'other', 'status': 'CNF', 'status_accuracy': 10}

{'name': 'Psybeam', 'type': 'Psychic', 'category': 'Special', 'base_power': 65, 'accuracy': 100, 'max_pp': 20, '

In [247]:
# Next, stat-changing moves
stat_moves = ['Acid Armor', 'Agility', 'Amnesia', 'Barrier', 'Defense Curl', 'Double Team',
              'Flash', 'Growl', 'Growth', 'Harden', 'Kinesis', 'Leer', 'Meditate', 'Minimize',
              'Sand-Attack', 'Screech', 'Sharpen', 'SmokeScreen', 'String Shot', 'Swords Dance',
              'Tail Whip', 'Withdraw']
n1 = len(stat_moves)
targets = (['self']*6 + ['other']*2 + ['self']*2 + ['other']*2 + ['self']*2 + 
           ['other']*2 + ['self'] + ['other']*2 + ['self'] + ['other'] + ['self'])
n2 = len(targets)
# Recall the mapping between stats: att=1, def=2, spe=3, spc=4, acc=5, eva=6
indices = [2, 3, 4, 2, 2, 6, 5, 1, 4, 2, 5, 2, 1, 6, 5, 2, 1, 5, 3, 1, 2, 2]
n3 = len(indices)
deltas = [2, 2, 2, 2, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -2, 1, -1, -1, 2, -1, 1]
n4 = len(deltas)
# Finally, copy main accuracy to stat accuracy

if n1 == n2 and n1 == n3 and n1 == n4:
    add_items(stat_moves, 'stat_target', targets)
    add_items(stat_moves, 'stat_index', indices)
    add_items(stat_moves, 'stat_delta', deltas)
    copy_vals(stat_moves, 'accuracy', 'stat_accuracy')

view_moves(stat_moves)

{'name': 'Acid Armor', 'type': 'Poison', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 40, 'stat_target': 'self', 'stat_index': 2, 'stat_delta': 2, 'stat_accuracy': '-'}

{'name': 'Agility', 'type': 'Psychic', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 30, 'stat_target': 'self', 'stat_index': 3, 'stat_delta': 2, 'stat_accuracy': '-'}

{'name': 'Amnesia', 'type': 'Psychic', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 20, 'stat_target': 'self', 'stat_index': 4, 'stat_delta': 2, 'stat_accuracy': '-'}

{'name': 'Barrier', 'type': 'Psychic', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 30, 'stat_target': 'self', 'stat_index': 2, 'stat_delta': 2, 'stat_accuracy': '-'}

{'name': 'Defense Curl', 'type': 'Normal', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 40, 'stat_target': 'self', 'stat_index': 2, 'stat_delta': 1, 'stat_accuracy': '-'}

{'name': 'Double Team', 'type': 'Normal', 'catego

In [249]:
# Next, damaging moves with secondary stat change effects
secondary_stat_moves = ['Acid', 'Aurora Beam', 'Bubble', 'BubbleBeam', 'Constrict', 'Psychic']
n1 = len(secondary_stat_moves)
targets = ['other']*6
n2 = len(targets)
# Recall the mapping between stats: att=1, def=2, spe=3, spc=4, acc=5, eva=6
indices = [2, 1, 3, 3, 3, 4]
n3 = len(indices)
deltas = [-1]*6
n4 = len(deltas)
accs = [33]*6
n5 = len(accs)

if n1 == n2 and n1 == n3 and n1 == n4 and n1 == n5:
    add_items(secondary_stat_moves, 'stat_target', targets)
    add_items(secondary_stat_moves, 'stat_index', indices)
    add_items(secondary_stat_moves, 'stat_delta', deltas)
    add_items(secondary_stat_moves, 'stat_accuracy', accs)

view_moves(secondary_stat_moves)

{'name': 'Acid', 'type': 'Poison', 'category': 'Physical', 'base_power': 40, 'accuracy': 100, 'max_pp': 30, 'stat_target': 'other', 'stat_index': 2, 'stat_delta': -1, 'stat_accuracy': 33}

{'name': 'Aurora Beam', 'type': 'Ice', 'category': 'Special', 'base_power': 65, 'accuracy': 100, 'max_pp': 20, 'stat_target': 'other', 'stat_index': 1, 'stat_delta': -1, 'stat_accuracy': 33}

{'name': 'Bubble', 'type': 'Water', 'category': 'Special', 'base_power': 20, 'accuracy': 100, 'max_pp': 30, 'stat_target': 'other', 'stat_index': 3, 'stat_delta': -1, 'stat_accuracy': 33}

{'name': 'BubbleBeam', 'type': 'Water', 'category': 'Special', 'base_power': 65, 'accuracy': 100, 'max_pp': 20, 'stat_target': 'other', 'stat_index': 3, 'stat_delta': -1, 'stat_accuracy': 33}

{'name': 'Constrict', 'type': 'Normal', 'category': 'Physical', 'base_power': 10, 'accuracy': 100, 'max_pp': 35, 'stat_target': 'other', 'stat_index': 3, 'stat_delta': -1, 'stat_accuracy': 33}

{'name': 'Psychic', 'type': 'Psychic', 'cat

In [261]:
# Finally, all the other unique moves
recoil_25pct = ['Double-Edge', 'Submission', 'Take Down']
recoil_50pct = ['Struggle']
crash = ['Hi Jump Kick', 'Jump Kick']
add_items_one_val(recoil_25pct, 'recoil', 0.25)
add_items_one_val(recoil_50pct, 'recoil', 0.50)
add_items_one_val(crash, 'crash', 1)

multihit_2x = ['Bonemerang', 'Double Kick', 'Twineedle']
multihit_random = ['Barrage', 'Comet Punch', 'DoubleSlap', 'Fury Attack', 
                   'Fury Swipes', 'Pin Missile', 'Spike Cannon']
add_items_one_val(multihit_2x, 'multihit', 2)
add_items_one_val(multihit_random, 'multihit', 'random')

hp_drain = ['Absorb', 'Dream Eater', 'Leech Life', 'Mega Drain']
add_items_one_val(hp_drain, 'hp_drain', 0.5)

recovery = ['Recover', 'Softboiled']
add_items_one_val(recovery, 'recovery', 0.5)

high_crit = ['Crabhammer', 'Karate Chop', 'Razor Leaf', 'Slash']
add_items_one_val(high_crit, 'high_crit', 8)

fixed_damage = ['Dragon Rage', 'Night Shade', 'Psywave', 'Seismic Toss', 'SonicBoom', 'Super Fang']
ohko = ['Fissure', 'Guillotine', 'Horn Drill']
add_item('Dragon Rage', 'fixed_damage', 40)
add_item('SonicBoom', 'fixed_damage', 20)
add_item('Night Shade', 'fixed_damage', 'level')
add_item('Seismic Toss', 'fixed_damage', 'level')
add_item('Psywave', 'fixed_damage', 'random')
add_item('Super Fang', 'fixed_damage', 'half_current')
add_items_one_val(ohko, 'ohko', True)


trapping = ['Bind', 'Clamp', 'Fire Spin', 'Wrap']
add_items_one_val(trapping, 'trapping', 'random')

other_unique = ['Counter', 'Dig', 'Explosion', 'Fly', 'Hyper Beam', 'Light Screen', 
                'Petal Dance', 'Quick Attack', 'Reflect', 'Selfdestruct', 'SolarBeam', 
                'Swift', 'Thrash', 'Toxic']
add_item('Quick Attack', 'priority', 1)
add_item('Counter', 'priority', -1)
add_item('Counter', 'fixed_damage', 'last_damage')
add_items_one_val(['Dig', 'Fly', 'SolarBeam'], '2_turn', True)
add_item('Dig', '2_turn_message', 'dug a hole!')
add_item('Fly', '2_turn_message', 'flew up high!')
add_item('SolarBeam', '2_turn_message', 'took in sunlight!')
add_items_one_val(['Explosion', 'Selfdestruct'], 'selfdestruct', True)
add_item('Hyper Beam', 'recharge', True)
add_item('Light Screen', 'lightscreen', True)
add_item('Reflect', 'reflect', True)
add_items_one_val(['Petal Dance', 'Thrash'], 'multiturn', 'random')
# should not need anything special for Swift or Toxic

implement_later = ['Conversion', 'Disable', 'Metronome', 'Mimic', 'Mirror Move', 'Substitute', 'Transform']
add_items_one_val(implement_later, 'implemented', False)

In [263]:
# Now let's check our work -- QA time!
# First print out all the unique keys -- will need this for implementing the MoveUser class
def get_all_keys():
    keys = set()
    for d_move in moves_dict.values():
        keys = keys.union(set(d_move.keys()))
    return sorted(keys)
print(f'List of all keys in moves_dict:\n{get_all_keys()}')

# Next let's check all the unique categories
def get_all_values(key):
    vals = set()
    for d_move in moves_dict.values():
        vals.add(d_move[key])
    return sorted(vals)
print(f'List of all categories:\n{get_all_values("category")}')
# And list all moves in the 'Unique' category to see if we missed anything,
# or if anything should be moved to a different category
def get_names_by_value(key, val):
    names = []
    for d_move in moves_dict.values():
        try:
            this_val = d_move[key]
        except KeyError:
            this_val = None
        if this_val == val:
            names.append(d_move['name'])
    return sorted(names)
def print_by_value(key, val):
    for d_move in moves_dict.values():
        try:
            this_val = d_move[key]
        except KeyError:
            this_val = None
        if this_val == val:
            print(d_move)
            print()

print(f'List of all moves with "Unique" category:\n{get_names_by_value("category", "Unique")}')
'''
A few things need to be fixed:
1. Disable, Toxic --> Status
2. Roar, Splash, Whirlwind --> need to implement later
3. Sand-Attack, SmokeScreen --> Stat
'''
add_items_one_val(['Disable', 'Toxic'], 'category', 'Status')
add_items_one_val(['Sand-Attack', 'SmokeScreen'], 'category', 'Stat')
implement_later += ['Roar', 'Splash', 'Whirlwind']
implement_later = sorted(implement_later)
add_items_one_val(implement_later, 'implemented', False)
print(f'Post-fix moves with "Unique" category:\n{get_names_by_value("category", "Unique")}')
print('More details:')
print_by_value('category', 'Unique')

# Now look at all moves with '-' power or accuracy
print(f'List of all moves with "-" base power:\n{get_names_by_value("base_power", "-")}')
print('More details:')
print_by_value('base_power', '-')
print()
print(f'List of all moves with "-" accuracy:\n{get_names_by_value("accuracy", "-")}')
print('More details:')
print_by_value('accuracy', '-')
print()

# Finally let's double-check Struggle
print(f'Details for Struggle:\n{moves_dict["Struggle"]}')

List of all keys in moves_dict:
['2_turn', '2_turn_message', 'accuracy', 'base_power', 'category', 'crash', 'fixed_damage', 'high_crit', 'hp_drain', 'implemented', 'lightscreen', 'max_pp', 'multihit', 'multiturn', 'name', 'ohko', 'priority', 'recharge', 'recoil', 'recovery', 'reflect', 'selfdestruct', 'stat_accuracy', 'stat_delta', 'stat_index', 'stat_target', 'status', 'status_accuracy', 'status_target', 'trapping', 'type']
List of all categories:
['Physical', 'Special', 'Stat', 'Status', 'Unique']
List of all moves with "Unique" category:
['Conversion', 'Light Screen', 'Metronome', 'Mimic', 'Mirror Move', 'Recover', 'Reflect', 'Roar', 'Softboiled', 'Splash', 'Substitute', 'Transform', 'Whirlwind']
Post-fix moves with "Unique" category:
['Conversion', 'Light Screen', 'Metronome', 'Mimic', 'Mirror Move', 'Recover', 'Reflect', 'Roar', 'Softboiled', 'Splash', 'Substitute', 'Transform', 'Whirlwind']
More details:
{'name': 'Conversion', 'type': 'Normal', 'category': 'Unique', 'base_power':

In [266]:
# Looks pretty good now... as a final check, let's look through the entire
# move list to make sure nothing is clearly wrong
for key, val in moves_dict.items():
    print(f'{key}: {val}')
    print()
'''
Some details need to be fixed:
Gust: Flying --> Normal
Karate Chop: Fighting --> Normal
And remember -- status moves ignore type immunities, except for Thunder Wave against Ground-types
'''
add_items_one_val(['Gust', 'Karate Chop'], 'type', 'Normal')

Absorb: {'name': 'Absorb', 'type': 'Grass', 'category': 'Special', 'base_power': 20, 'accuracy': 100, 'max_pp': 20, 'hp_drain': 0.5}

Acid: {'name': 'Acid', 'type': 'Poison', 'category': 'Physical', 'base_power': 40, 'accuracy': 100, 'max_pp': 30, 'stat_target': 'other', 'stat_index': 2, 'stat_delta': -1, 'stat_accuracy': 33}

Acid Armor: {'name': 'Acid Armor', 'type': 'Poison', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 40, 'stat_target': 'self', 'stat_index': 2, 'stat_delta': 2, 'stat_accuracy': '-'}

Agility: {'name': 'Agility', 'type': 'Psychic', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 30, 'stat_target': 'self', 'stat_index': 3, 'stat_delta': 2, 'stat_accuracy': '-'}

Amnesia: {'name': 'Amnesia', 'type': 'Psychic', 'category': 'Stat', 'base_power': '-', 'accuracy': '-', 'max_pp': 20, 'stat_target': 'self', 'stat_index': 4, 'stat_delta': 2, 'stat_accuracy': '-'}

Aurora Beam: {'name': 'Aurora Beam', 'type': 'Ice', 'category': 'Special

For convenience later on in the main program, as a final step, we'll ensure that all moves have the same set of keys. This will let us do things like
```
if move.recoil:
    user.hp -= damage*move.recoil
```
instead of
```
if 'recoil' in move.dict.keys():
    user.hp -= damage*move.dict['recoil']
```

In [280]:
all_keys = get_all_keys()
for name, d in moves_dict.items():
    for key in all_keys:
        if key not in d.keys():
            d[key] = None
for key, val in moves_dict.items():
    print(f'{key}: {val}')
    print()

Absorb: {'name': 'Absorb', 'type': 'Grass', 'category': 'Special', 'base_power': 20, 'accuracy': 100, 'max_pp': 20, 'hp_drain': 0.5, '2_turn': None, '2_turn_message': None, 'crash': None, 'fixed_damage': None, 'high_crit': None, 'implemented': None, 'lightscreen': None, 'multihit': None, 'multiturn': None, 'ohko': None, 'priority': None, 'recharge': None, 'recoil': None, 'recovery': None, 'reflect': None, 'selfdestruct': None, 'stat_accuracy': None, 'stat_delta': None, 'stat_index': None, 'stat_target': None, 'status': None, 'status_accuracy': None, 'status_target': None, 'trapping': None}

Acid: {'name': 'Acid', 'type': 'Poison', 'category': 'Physical', 'base_power': 40, 'accuracy': 100, 'max_pp': 30, 'stat_target': 'other', 'stat_index': 2, 'stat_delta': -1, 'stat_accuracy': 33, '2_turn': None, '2_turn_message': None, 'crash': None, 'fixed_damage': None, 'high_crit': None, 'hp_drain': None, 'implemented': None, 'lightscreen': None, 'multihit': None, 'multiturn': None, 'ohko': None, '

# `moves_dict` is now complete!
... minus a few unimplemented unique moves, which we'll deal with later

Next, we need to save the dict to a file, and test out re-reading it from the file.

In [267]:
import json

In [281]:
moves_filename = 'data/moves.json'
with open(moves_filename, 'w') as f:
    json.dump(moves_dict, f)

# now re-read it back to make sure everything is preserved correctly
with open(moves_filename) as f:
    md2 = json.load(f)
for key, val in md2.items():
    print(f'{key}: {val}')
    print()
# Looks perfect!

Absorb: {'name': 'Absorb', 'type': 'Grass', 'category': 'Special', 'base_power': 20, 'accuracy': 100, 'max_pp': 20, 'hp_drain': 0.5, '2_turn': None, '2_turn_message': None, 'crash': None, 'fixed_damage': None, 'high_crit': None, 'implemented': None, 'lightscreen': None, 'multihit': None, 'multiturn': None, 'ohko': None, 'priority': None, 'recharge': None, 'recoil': None, 'recovery': None, 'reflect': None, 'selfdestruct': None, 'stat_accuracy': None, 'stat_delta': None, 'stat_index': None, 'stat_target': None, 'status': None, 'status_accuracy': None, 'status_target': None, 'trapping': None}

Acid: {'name': 'Acid', 'type': 'Poison', 'category': 'Physical', 'base_power': 40, 'accuracy': 100, 'max_pp': 30, 'stat_target': 'other', 'stat_index': 2, 'stat_delta': -1, 'stat_accuracy': 33, '2_turn': None, '2_turn_message': None, 'crash': None, 'fixed_damage': None, 'high_crit': None, 'hp_drain': None, 'implemented': None, 'lightscreen': None, 'multihit': None, 'multiturn': None, 'ohko': None, '

# Move list modifications
There are some moves we definitely don't want to implement, so let's remove them from all Pokemons' movepools. We also should ideally convert the pandas DataFrame to a dict stored in json, like we now do for moves. While we're at it, let's also convert the movepools to lists.

In [282]:
# First aggregate all the implemented moves into a list
good_moves = list(moves_dict.keys())
# Then convert the species DF into a dict
species_dict = {}
def convert_species():
    species_dict.clear()
    for i, row in df_all.iterrows():
        name = row['name']
        this_species = {}
        this_species['name'] = name
        this_species['base_hp'] = row['base_hp']
        this_species['base_att'] = row['base_att']
        this_species['base_def'] = row['base_def']
        this_species['base_spe'] = row['base_spe']
        this_species['base_spc'] = row['base_spc']
        this_species['type1'] = row['type1']
        this_species['type2'] = row['type2']
        this_species['randomizer_level'] = row['randomizer_level']
        movepool_str = row['movepool']
        movepool_list = movepool_str.split(';')
        good_movepool = []
        for move in movepool_list:
            if move in good_moves:
                good_movepool.append(move)
        this_species['movepool'] = good_movepool
        species_dict[name] = this_species

convert_species()
for key, val in species_dict.items():
    print(f'{key}: {val}')
    print()

Bulbasaur: {'name': 'Bulbasaur', 'base_hp': 45, 'base_att': 49, 'base_def': 49, 'base_spe': 45, 'base_spc': 65, 'type1': 'Grass', 'type2': 'Poison', 'randomizer_level': 89, 'movepool': ['Body Slam', 'Cut', 'Double Team', 'Double-Edge', 'Growl', 'Growth', 'Leech Seed', 'Mega Drain', 'Mimic', 'PoisonPowder', 'Razor Leaf', 'Reflect', 'Rest', 'Sleep Powder', 'SolarBeam', 'Substitute', 'Swords Dance', 'Tackle', 'Take Down', 'Toxic', 'Vine Whip']}

Ivysaur: {'name': 'Ivysaur', 'base_hp': 60, 'base_att': 62, 'base_def': 63, 'base_spe': 60, 'base_spc': 80, 'type1': 'Grass', 'type2': 'Poison', 'randomizer_level': 80, 'movepool': ['Body Slam', 'Cut', 'Double Team', 'Double-Edge', 'Growl', 'Growth', 'Leech Seed', 'Mega Drain', 'Mimic', 'PoisonPowder', 'Razor Leaf', 'Reflect', 'Rest', 'Sleep Powder', 'SolarBeam', 'Substitute', 'Swords Dance', 'Tackle', 'Take Down', 'Toxic', 'Vine Whip']}

Venusaur: {'name': 'Venusaur', 'base_hp': 80, 'base_att': 82, 'base_def': 83, 'base_spe': 80, 'base_spc': 100,

In [283]:
# Now save the species dict to a json file
species_filename = 'data/species.json'
with open(species_filename, 'w') as f:
    json.dump(species_dict, f)

# now re-read it back to make sure everything is preserved correctly
with open(species_filename) as f:
    sd2 = json.load(f)
for key, val in sd2.items():
    print(f'{key}: {val}')
    print()
# Looks perfect!

Bulbasaur: {'name': 'Bulbasaur', 'base_hp': 45, 'base_att': 49, 'base_def': 49, 'base_spe': 45, 'base_spc': 65, 'type1': 'Grass', 'type2': 'Poison', 'randomizer_level': 89, 'movepool': ['Body Slam', 'Cut', 'Double Team', 'Double-Edge', 'Growl', 'Growth', 'Leech Seed', 'Mega Drain', 'Mimic', 'PoisonPowder', 'Razor Leaf', 'Reflect', 'Rest', 'Sleep Powder', 'SolarBeam', 'Substitute', 'Swords Dance', 'Tackle', 'Take Down', 'Toxic', 'Vine Whip']}

Ivysaur: {'name': 'Ivysaur', 'base_hp': 60, 'base_att': 62, 'base_def': 63, 'base_spe': 60, 'base_spc': 80, 'type1': 'Grass', 'type2': 'Poison', 'randomizer_level': 80, 'movepool': ['Body Slam', 'Cut', 'Double Team', 'Double-Edge', 'Growl', 'Growth', 'Leech Seed', 'Mega Drain', 'Mimic', 'PoisonPowder', 'Razor Leaf', 'Reflect', 'Rest', 'Sleep Powder', 'SolarBeam', 'Substitute', 'Swords Dance', 'Tackle', 'Take Down', 'Toxic', 'Vine Whip']}

Venusaur: {'name': 'Venusaur', 'base_hp': 80, 'base_att': 82, 'base_def': 83, 'base_spe': 80, 'base_spc': 100,

# Final notes
There are still (as of 11/18/25) quite a few items to be addressed in the main code. A maybe(?) complete list:
1. Edit `globals.py` to retrieve the json files and populate species and moves.
2. Edit `pokemon.py` and `move.py` to make their initializers use the new dicts
3. Create a new `MoveUser` class to handle all the new special cases and separate the logic from the `Battle` class. Speaking of new special cases...
    - First, make sure damage, status, and stat change moves are all still handled correctly
    - 2-turn moves ('2_turn', '2_turn_message')
    - Crash damage ('crash')
    - Fixed damage ('fixed_damage')
    - High crit rate ('high_crit')
    - HP-drain ('hp_drain')
    - Light Screen/Reflect ('lightscreen', 'reflect')
    - Multi-hit ('multihit')
    - Petal Dance/Thrash ('multiturn')
    - Quick Attack/Counter ('priority')
    - Hyper Beam ('recharge')
    - Recoil ('recoil')
    - Recovery ('recovery')
    - Explosion/Seldestruct ('selfdestruct')
    - Trapping ('trapping')
4. Handle the last few unimplemented moves at some point. Namely:
    - Conversion
    - Disable
    - Metronome
    - Mimic
    - Mirror Move
    - Roar
    - Splash
    - Substitute
    - Transform
    - Whirlwind